## Import Modules

In [1]:
import pandas as pd
import numpy as np
import os

## Import Data

In [2]:
path = r'C:\Users\david\04-06-2024 Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_combined.pkl'))

In [4]:
ords_prods_merge.shape

(32404859, 18)

In [5]:
df = ords_prods_merge[:1000000]

In [6]:
df.shape

(1000000, 18)

In [7]:
ords_prods_merge

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_ordinal_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0,both,195,Soda,77,7,9.0
1,1,1,2398795,1,2,3,7,15.0,False,196,1,1,both,195,Soda,77,7,9.0
2,2,2,473747,1,3,3,12,21.0,False,196,1,1,both,195,Soda,77,7,9.0
3,3,3,2254736,1,4,4,7,29.0,False,196,1,1,both,195,Soda,77,7,9.0
4,4,4,431534,1,5,4,15,28.0,False,196,1,1,both,195,Soda,77,7,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,3360493,3360493,1320836,202557,17,2,15,1.0,False,43553,2,1,both,43557,Orange Energy Shots,64,7,3.7
32404855,3360494,3360494,31526,202557,18,5,11,3.0,False,43553,2,1,both,43557,Orange Energy Shots,64,7,3.7
32404856,3376038,3376038,758936,203436,1,2,7,NaN,True,42338,4,0,both,42342,"Zucchini Chips, Pesto",50,19,6.9
32404857,3376039,3376039,2745165,203436,2,3,5,15.0,False,42338,16,1,both,42342,"Zucchini Chips, Pesto",50,19,6.9


In [8]:
def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range product'
    else:
        return 'Not enough data'

## Price Label Column

In [9]:
df['price_range'] = df.apply(price_label, axis = 1)

C:\Users\david\AppData\Local\Temp\ipykernel_9672\935325922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis = 1)


In [10]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    756450
Low-range product    243550
Name: count, dtype: int64

In [11]:
df['prices'].max()

14.8

In [12]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [13]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [14]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [15]:
ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

## Busiest Day Column

In [16]:
result = []

for value in ords_prods_merge["order_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [17]:
ords_prods_merge['busiest_day'] = result

In [18]:
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

## Question 2.

In [19]:
#Calculating busiest days
ords_prods_merge['order_day_of_week'].value_counts(dropna = False)

order_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [20]:
#flags if order was during top 2 busiest days or bottom two busiest days
result = []

for value in ords_prods_merge["order_day_of_week"]:
  if value == 0 or value == 1:
    result.append("Busiest days")
  elif value == 4 or value == 3:
    result.append("Least busy days")
  else:
    result.append("Regularly busy")

In [21]:
ords_prods_merge['busiest_day'] = result

# Question 3

In [22]:
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy     12916111
Busiest days       11864412
Least busy days     7624336
Name: count, dtype: int64

# Question 4

In [23]:
ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

In [24]:
busiest_hours = []

for x in ords_prods_merge['order_hour_of_day']:
    if x == 10 or x == 11 or x == 14 or x == 15 or x == 13:
        busiest_hours.append('Most orders - Top 5 Hours')
    elif x == 3 or x == 4 or x == 2 or x == 5 or x == 1:
        busiest_hours.append('Fewest Orders - Bottom 5 Hours')
    else:
        busiest_hours.append('Average Orders')

In [25]:
ords_prods_merge['busiest_period_of_day'] = busiest_hours

# Question 5

In [26]:
ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Average Orders                    18517166
Most orders - Top 5 Hours         13510134
Fewest Orders - Bottom 5 Hours      377559
Name: count, dtype: int64

# Question 7 - Exporting as PKL

In [27]:
ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge_new_columns.pkl'))